In [28]:
%load_ext autoreload
%autoreload 2
from ai_cdss.data_loader import DataLoader
from ai_cdss.data_processor import DataProcessor

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [29]:
# import this
from ai_cdss.cdss import CDSS

In [22]:
patient_list = [
    788
]

loader = DataLoader("app")
processor = DataProcessor([1,1,1])
timeseries = loader.load_timeseries_data(
    patient_list
)
session = loader.load_session_data(
    patient_list
)
ppf = loader.load_ppf_data(
    patient_list
)
# scoring = processor.process_data(data, ppf)

# cdss = CDSS(
#     scoring
# )

Database engine created successfully
Database engine closed
Database engine created successfully
Database engine closed
Database engine created successfully


INFO:ai_cdss.data_loader:Session data loaded successfully.
INFO:ai_cdss.data_loader:PPF data loaded successfully.


Database engine closed


In [23]:
session

,PATIENT_ID,HOSPITAL_ID,PARETIC_SIDE,UPPER_EXTREMITY_TO_TRAIN,HAND_RAISING_CAPACITY,COGNITIVE_FUNCTION_LEVEL,HAS_HEMINEGLIGENCE,GENDER,SKIN_COLOR,AGE,...,PROTOCOL_TYPE,AR_MODE,WEEKDAY_INDEX,REAL_SESSION_DURATION,PRESCRIBED_SESSION_DURATION,SESSION_DURATION,ADHERENCE,TOTAL_SUCCESS,TOTAL_ERRORS,SCORE
0,788,40,LEFT,LEFT,HIGH,MEDIUM,0,MALE,6B4731,44,...,Hands,NONE,0,399,300,300,1.0,57,107,57
1,788,40,LEFT,LEFT,HIGH,MEDIUM,0,MALE,6B4731,44,...,Body,NONE,0,329,300,300,1.0,19,0,19
2,788,40,LEFT,LEFT,HIGH,MEDIUM,0,MALE,6B4731,44,...,AR,TABLE,0,365,300,300,1.0,0,0,15
3,788,40,LEFT,LEFT,HIGH,MEDIUM,0,MALE,6B4731,44,...,AR,TABLE,0,346,300,300,1.0,0,0,0
4,788,40,LEFT,LEFT,HIGH,MEDIUM,0,MALE,6B4731,44,...,Hands,NONE,1,385,300,300,1.0,69,10,96
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
152,788,40,LEFT,LEFT,HIGH,MEDIUM,0,MALE,6B4731,44,...,AR,TABLE,3,400,240,240,1.0,33,12,53
153,788,40,LEFT,LEFT,HIGH,MEDIUM,0,MALE,6B4731,44,...,AR,TABLE,3,334,300,300,1.0,40,5,82
154,788,40,LEFT,LEFT,HIGH,MEDIUM,0,MALE,6B4731,44,...,Hands,NONE,3,66,300,28,0.093333,5,9,5
155,788,40,LEFT,LEFT,HIGH,MEDIUM,0,MALE,6B4731,44,...,Hands,NONE,3,21,300,8,0.026667,0,0,0


In [21]:
from ai_cdss.models import TimeseriesSchema

TimeseriesSchema.example(num_patients=1, num_sessions_per_patient=10, num_protocols=3)

,PATIENT_ID,SESSION_ID,PROTOCOL_ID,GAME_MODE,SECONDS_FROM_START,DM_KEY,DM_VALUE,PE_KEY,PE_VALUE
0,1,1,1,hard,0,param_4,9.51,metric_1,77.97
1,1,1,1,hard,10,param_1,1.56,metric_1,10.00
2,1,1,1,hard,20,param_3,8.66,metric_2,14.29
3,1,1,1,hard,30,param_3,0.21,metric_2,72.20
4,1,1,1,hard,40,param_2,2.12,metric_2,99.22
...,...,...,...,...,...,...,...,...,...
145,1,10,3,hard,0,param_2,4.93,metric_1,40.52
146,1,10,3,hard,10,param_2,0.56,metric_2,76.72
147,1,10,3,hard,20,param_3,6.49,metric_1,68.10
148,1,10,3,hard,30,param_4,9.62,metric_1,47.77


In [ ]:
data.groupby(["PATIENT_ID","PRESCRIPTION_ID", "PROTOCOL_ID"])[["WEEKDAY_INDEX","PRESCRIPTION_ENDING_DATE"]].last()

In [ ]:
data["ADHERNCE"] = (
    data.groupby("SESSION_ID")["ADHERENCE"]
    .transform(lambda x: x.ewm(alpha=0.5, adjust=True).mean())
)

In [6]:
import pandas as pd
from pandera import Check, Column, DataFrameSchema

schema = DataFrameSchema(
    {
        "Name": Column(
            str,
            [
                Check.str_matches("[A-Za-z0-9_]+$"),
                Check.str_length(min_value=1, max_value=25),
            ],
        ),
        "Year": Column(
            int,
            Check.in_range(min_value=1947, max_value=3000),
        ),
        "Extract": Column(str, Check.isin(["A", "B", "C"])),
        "Start": Column(int, Check.in_range(1, 1000)),
        "Length": Column(int, Check.in_range(0, 50)),
    },
    unique=["Name", "Year", "Extract"],
)

# This works
schema.example(size=3)

,Name,Year,Extract,Start,Length
0,e9Is,2509,A,939,26
1,6VEGPh,2263,C,712,36
2,kC,2971,B,562,8


In [ ]:
from rgs_interface.data.interface import fetch_rgs_data

data = fetch_rgs_data(
    [775],
    "app"
)

In [ ]:
data

In [ ]:
data.dtypes

In [ ]:
data.dtypes

In [ ]:
processor.data

In [ ]:
aggregated_data

In [ ]:
aggregated_data

In [ ]:
data.WEEKDAY_INDEX

In [24]:
metrics  = pd.read_csv("../data/metric_plus.csv")

In [27]:
metrics.METRIC_KEY.unique()

array(['totalDisplacement', 'volume', 'surfaceArea', 'area(x,y,z)',
       'minimumPoint(x,y,z)', 'maximumPoint(x,y,z)'], dtype=object)